# 1. MNIST dataset

In de eerste opdracht trainen we een simpel neuraal netwerk met Keras. De dataset die we hiervoor gebruiken is MNIST. Deze dataset bestaat uit grijswaarde plaatjes van cijfers. Elk plaatje is 28×28 pixels.

## Over deze tutorial

Deze tutorial maakt gebruik van [Jupyter notebooks](http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb). Wat je moet weten voor nu:
* De programeertaal is Python 3
* Een notebook bestaat uit meerdere cellen onder elkaar.
* Voer de cellen uit van boven naar beneden, en pas aan of voeg cellen toe waar nodig.
* Voor het uitvoeren van een cel: `Shift + Enter`
* Voor het toevoegen van cellen: druk op de knop `[+] Code`.

## Bibliotheken

Voordat we kunnen beginnen moeten een aantal bibliotheken worden geïmporteerd.

In [ ]:
# We gebruiken Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# En numpy voor het werken met arrays
import numpy as np
# pyplot voor het tekenen van plaatjes en grafieken.
import matplotlib.pyplot as plt
%matplotlib inline

## Dataset inladen
Eerst zullen we de dataset inladen. De MNIST dataset is standaard beschikbaar in Keras.

In [ ]:
from keras.datasets import mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Shape of the data
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test  = x_test.reshape(x_test.shape[0],   img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# Normalize images
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train_mat = keras.utils.to_categorical(y_train, num_classes)
y_test_mat = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Toon de eerste paar plaatjes uit de training set, en de bijbehorende labels
plt.figure(None,(18,18))
for i in range(10):
  plt.subplot(1,10,i+1)
  plt.imshow(x_train[i].reshape(28,28),"Greys") # Toon x_train[i]
plt.show()
# De bijbehorende labels
print("Labels:",y_train[0:10])

## Een model bouwen en trainen
Nu bouwen we een keras model, en trainen dit op de training set.

Let op dat GPU acceleration ingeschakeld staat, anders duurt dit trainen erg lang (probeer maar eens).

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train 10 epochs met batches van 128 samples
batch_size = 128
epochs = 10

history = model.fit(x_train, y_train_mat,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test_mat))

# Hoe goed werkt het model op de testset
score = model.evaluate(x_test, y_test_mat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Plot van de accuracy tijdens het trainen
plt.plot(history.epoch,history.history['acc'],label="Training accuracy")
plt.plot(history.epoch,history.history['val_acc'],label="Validation accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

plt.plot(history.epoch,history.history['loss'],label="Training loss")
plt.plot(history.epoch,history.history['val_loss'],label="Validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

## Vragen
Probeer deze vragen te beantwoorden. Hiervoor moet de bovenstaande code misschien worden aangepast. Eventueel kunnen onderaan dit document extra cellen worden toegevoegd. Tip: je kan een notebook ook gebruiken als rekenmachine.

1. De MNIST dataset bestaat uit 60000 plaatjes voor de trainingset en 10000 voor de testset. Kijk naar de accuracy van het model op de testset. Hoeveel plaatjes worden fout geclassificeerd?

1. Uit welke lagen bestaat het model? Kan je het model simpeler maken door sommige lagen weg te laten? Gaat de accuracy hierdoor omlaag?

1. Kijk naar de training loss. Heeft het zin om langer door te gaan met trainen? Je kunt dit evt. ook uitproberen.

## Foute classificaties
Het neurale netwerk maak maar weinig fouten. We kunnen deze verder bekijken om te zien of deze voor mensen ook lastig te classificeren zijn.

Tot nu toe hebben we alleen gekeken naar de accuracy. Om de fouten te vinden moeten we de daadwerkelijke voorspellingen hebben.

In [ ]:
# Voorspel de labels op de testset volgens het model
predicted_test = model.predict(x_test, batch_size=batch_size)

# Het meest waarschijnlijke label
best_predicted_test = np.argmax(predicted_test,1)

In [ ]:
# Indices waar het model de fout in gaat 
wrong = np.argwhere(best_predicted_test != y_test)[:,0]

In [ ]:
# Weergave van een aantal fout voorspelde plaatjes
plt.figure(None,(18,18))
for i in range(10):
  plt.subplot(1,10,i+1)
  plt.imshow(x_test[wrong[i]].reshape(28,28),"Greys")
plt.show()
print("Echte labels     ", y_test[wrong[0:10]])
print("Voorspelde labels", best_predicted_test[wrong[0:10]])

## Vragen

4. `predicted_test` is een matrix, waar elke rij de voorspelde kans aangeeft dat het bijboherende plaatje in een bepaalde klasse valt. Hierboven is alleen de meest waarschijnlijke klasse gegeven als "voorspelde label". Wat is de bijbeherende kans? Is het model zeker van zijn zaak, of wordt de tweede keuze ook waarschijnlijk geacht?